<h1 align="center">Automatic Cover Letter Generation</h1>

___

This document aims to prove the feasibility of setting up an automated tool for generating cover letters based on a Linkedin job offer link and the user's CV description. (WIP)

# Loading Packages

In [13]:
from bs4 import BeautifulSoup
import requests
import json
import os
from mistralai.client import MistralClient# Import the MistralClient from the mistralai.client module
from mistralai.models.chat_completion import ChatMessage # Import the ChatMessage from the mistralai.models.chat_completion module
from dotenv import load_dotenv

# Extracting Data from Linkedin Website

In [3]:
test = requests.get('https://www.linkedin.com/jobs/search?keywords=Analyste+De+Donn%C3%A9es&location=Gen%C3%A8ve%2C+Gen%C3%A8ve%2C+Suisse&geoId=106633403&trk=public_jobs_jobs-search-bar_search-submit')
content = test.text

soup = BeautifulSoup(content, 'html.parser')

# List of Job Offers

In [4]:
list = soup.find_all('a', {'class':'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]'})

for i, item in enumerate(list):
    list[i] = item.get('href')
print(list[1])

https://fr.linkedin.com/jobs/view/analyste-de-donn%C3%A9es-pour-enseigner-les-math%C3%A9matiques-h-f-at-voscours-fr-3858720998?position=2&pageNum=0&refId=vVo9%2B2gTiilAm9Otm4rl%2BA%3D%3D&trackingId=bAg7saHsQUGPkBtJCHfVxQ%3D%3D&trk=public_jobs_jserp-result_search-card


Now that we have the list of links, we will retrieve the job description.

In [7]:
description = []

for i, link in enumerate(list):
    Job = requests.get(link)
    content = Job.text
    description.append(BeautifulSoup(content, 'html.parser').find('div', {'class':'show-more-less-html__markup'}))

Now that we have the list of descriptions, we will start by listing the expected skills for each job.

In [20]:
load_dotenv()
model = "mistral-large-latest"
api_key= os.getenv('MISTRAL_API_KEY')
client = MistralClient(api_key=api_key)
responses = []

for job in description[:3]:

    messages = [
        ChatMessage(role="system", content="tu es un programme qui liste les compétences à avoir pour répondre à une offre de travail, sur base de l'input que l'on te fournis, contenant la liste des comptétences à avoir, et tu ne répond qu'en bullet point"),
        ChatMessage(role="user", content=job.text.strip()),
    ]
    responses.append(client.chat(model=model, messages=messages))


In [33]:
print(responses[0].choices[0].message.content)

- Formation supérieure ou universitaire dans le domaine de la data
- Expérience réussie d'au moins 3 ans dans le domaine de l'analyse de données
- Connaissance du domaine RH et des outils informatiques liés, un atout
- Maîtrise des outils informatiques usuels et en particulier SAP, Tableau, Excel, outils d’extraction de données (Tableau Prep)
- Sens aigu de la confidentialité et de la discrétion
- Capacité à gérer les priorités avec une approche méthodologique
- Esprit d’analyse et de synthèse
- Bonne communication, à l'écoute et orienté clients
- Capacité à élaborer et mettre en place une feuille de route pour le reporting RH
- Capacité à collaborer avec les équipes métier pour comprendre leurs besoins et les traduire en indicateurs clés
- Capacité à collaborer avec la division Informatique pour automatiser les extractions de données et enrichir les sources de données
- Capacité à participer aux demandes internes et externes liées aux statistiques et reporting sur les données du perso

# Cover Letter Generation

In [44]:
with open('data.json', 'r') as file:
    data = json.load(file)

input = "liste des compétence à avoir pour le poste de data analyste: " + responses[0].choices[0].message.content + " liste des compétence du candidat : " + str(data)
messages1 = [
    ChatMessage(role="system", content="tu es un secretaire qui écrit un corps de lettre de motivation en croisant la liste des compétences à avoir que l'on te donne, avec la liste des compétences du candidat. Tu ne retournes uniquement que le corps de la lettre de motivation"),
    ChatMessage(role='user', content=input)
]
response = client.chat(model=model, messages=messages1)
print(response.choices[0].message.content)

Madame, Monsieur,

C'est avec un grand intérêt que je vous soumets ma candidature pour le poste de Data Analyste au sein de votre entreprise. Titulaire d'un Master en Financial Risk Engineering de l'ISFA et d'un Bachelor en Applied Mathematics in Social Sciences de l'Université Claude Bernard Lyon 1, j'ai acquis une solide formation dans le domaine de la data.

Mon expérience professionnelle de plus de trois ans en tant que Data Scientist / Data Analyst chez EXSOLVAE et en tant qu'Actuarial Analyst chez Ethias Insurance m'a permis de développer une expertise dans l'analyse de données. Durant ces expériences, j'ai eu l'occasion de développer et d'automatiser des outils d'analyse et de visualisation de données, ainsi que de participer à des projets de machine learning.

Ma connaissance du domaine RH et ma maîtrise des outils informatiques usuels, en particulier SAP, Tableau, Excel, et les outils d’extraction de données (Tableau Prep), me permettront de répondre efficacement aux besoins d